# Motivation

- *What is your dataset?*



- *Why did you choose this/these particular dataset(s)?*



- *What was your goal for the end user's experience?*


# Basic Stats

- *Write about your choices in data cleaning and preprocessing*.


#### **Data Cleaning**

**Which columns can we remove?**


**Which rows of columns should we remove?**

- *Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.*


# Data Analysis

- *Describe your data analysis and explain what you've learned about the dataset.*
- *If relevant, talk about your machine-learning.*

##### **FOR CLARITY OF THE EXPLAINER, THE DATA ANALYSIS IS DONE IN SEPARATE NOTEBOOKS AND REFERENCED. ONLY SELECT PLOTS ARE SHOWN HERE**

The data analysis plots are done in different notebooks as they are too large to have in one notebook. Selected plots are shown in this notebook to give an idea of the most important findings.
We know that you asked for the explainer notebook to contain all the code that made up this project but we truly believe that using our current folder structure with hyperlinks and reference links to the respective files are much manageable and clear to individuals grading this project

### Temporal Data Analysis

The full temporal data analysis is done in file [Temporal Data Analysis Code](data_analysis_and_preprocessing/temporal_plots.ipynb). Firstly, an analysis of all temporal changes for all top 25 most common crimes is done by Year, Month, Weekday, 24-Hour using multipanel bar plots. Here is an example of the Weekday and 24 plots as these were most relevant for travelling in NYC, since the yearly plot is hard to act upon, whereas a more managable hourly and weekday plot can be used to avoid crime hotspots. 

### Bar Plots

### Line And Polar Plots

These plots illustrate that there are significant temporal changes across all 

## Socioeconomic Data Analysis

# Genre

Which genre of data story did you use?

- Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
- Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?

## Visual Narrative

#### **Visual Structuring**

#### **Highlighting**


#### **Transition Guidance**



## Narrative Structure

#### **Ordering**


#### **Interactivity**


#### **Messaging**





# Visualizations
- Explain the visualizations you've chosen.
- Why are they right for the story you want to tell?

##### **Bar Charts**


##### **Line & Polar Plot**


##### **Bokeh Plots**


# Discussion
- What went well?,
- What is still missing? What could be improved?, Why?

### What went well?,
- t
- h

### What is still missing? What could be improved?, Why?
- o
- i


# Contributions

You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).
It is not OK simply to write "All group members contributed equally".

##### **General Notes on collaboration**

All 3 students have contributed equally to this project and we have assisted each other and understand each component of the final result. 

However, as required by DTU the following is an outline of the primary responsibilities for each student: